# portfolio

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit, f8, i8, b1, optional
import empyrical

In [3]:
from datetime import datetime
index = pd.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5),
    datetime(2020, 1, 6),
    datetime(2020, 1, 7),
])
price = pd.DataFrame({
    'a': [1, 2, 3, 4, 5, 6, 7],
    'b': [9, 8, 7, 6, 5, 4, 3],
    'c': [1, 2, 3, 4, 3, 2, 1],
    'd': [4, 3, 2, 1, 2, 3, 4]
}, index=index).astype(float)

print(price)

              a    b    c    d
2020-01-01  1.0  9.0  1.0  4.0
2020-01-02  2.0  8.0  2.0  3.0
2020-01-03  3.0  7.0  3.0  2.0
2020-01-04  4.0  6.0  4.0  1.0
2020-01-05  5.0  5.0  3.0  2.0
2020-01-06  6.0  4.0  2.0  3.0
2020-01-07  7.0  3.0  1.0  4.0


In [4]:
big_price = pd.DataFrame(np.random.uniform(size=(1000, 1000)).astype(float))
big_price.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]

print(big_price.shape)

(1000, 1000)


In [5]:
entries = pd.DataFrame.vbt.signals.generate_random(
    price.shape, 2, min_space=1, seed=42, index=price.index, columns=price.columns)
big_entries = pd.DataFrame.vbt.signals.generate_random(
    big_price.shape, 100, min_space=1, seed=42, index=big_price.index)

print(entries)

                a      b      c      d
2020-01-01  False  False   True  False
2020-01-02  False  False  False  False
2020-01-03   True   True  False   True
2020-01-04  False  False  False  False
2020-01-05   True   True  False  False
2020-01-06  False  False  False  False
2020-01-07  False  False   True   True


In [6]:
exits = entries.vbt.signals.generate_random_after(1, seed=42)
big_exits = big_entries.vbt.signals.generate_random_after(1, seed=42)

print(exits)

                a      b      c      d
2020-01-01  False  False  False  False
2020-01-02  False  False   True  False
2020-01-03  False  False  False  False
2020-01-04   True   True  False  False
2020-01-05  False  False  False   True
2020-01-06  False   True  False  False
2020-01-07   True  False  False  False


In [7]:
vbt.defaults.portfolio['init_capital'] = 100 # set init_capital in cash globally

In [8]:
# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.defaults.caching = False

## from_signals

In [9]:
portfolio = vbt.Portfolio.from_signals(price['a'], entries['a'], exits['a'], size=1)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price.iloc[:, 0], big_entries.iloc[:, 0], big_exits.iloc[:, 0])

   col  idx  size  price  fees  side
0    0    2   1.0    3.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    0    4   1.0    5.0   0.0     0
3    0    6   1.0    7.0   0.0     1
2020-01-01    0.0
2020-01-02    0.0
2020-01-03    1.0
2020-01-04    0.0
2020-01-05    1.0
2020-01-06    1.0
2020-01-07    0.0
Name: a, dtype: float64
2020-01-01    100.0
2020-01-02    100.0
2020-01-03     97.0
2020-01-04    101.0
2020-01-05     96.0
2020-01-06     96.0
2020-01-07    103.0
Name: a, dtype: float64
10.1 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1)

    col  idx  size  price  fees  side
0     0    2   1.0    3.0   0.0     0
1     0    3   1.0    4.0   0.0     1
2     0    4   1.0    5.0   0.0     0
3     0    6   1.0    7.0   0.0     1
4     1    2   1.0    7.0   0.0     0
5     1    3   1.0    6.0   0.0     1
6     1    4   1.0    5.0   0.0     0
7     1    5   1.0    4.0   0.0     1
8     2    0   1.0    1.0   0.0     0
9     2    1   1.0    2.0   0.0     1
10    2    6   1.0    1.0   0.0     0
11    3    2   1.0    2.0   0.0     0
12    3    4   1.0    2.0   0.0     1
13    3    6   1.0    4.0   0.0     0
              a    b    c    d
2020-01-01  0.0  0.0  1.0  0.0
2020-01-02  0.0  0.0  0.0  0.0
2020-01-03  1.0  1.0  0.0  1.0
2020-01-04  0.0  0.0  0.0  1.0
2020-01-05  1.0  1.0  0.0  0.0
2020-01-06  1.0  0.0  0.0  0.0
2020-01-07  0.0  0.0  1.0  1.0
                a      b      c      d
2020-01-01  100.0  100.0   99.0  100.0
2020-01-02  100.0  100.0  101.0  100.0
2020-01-03   97.0   93.0  101.0   98.0
2020-01-04  101.0   99.0  

In [11]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=np.inf) # all in/out
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=np.inf)

    col  idx        size  price  fees  side
0     0    2   33.333333    3.0   0.0     0
1     0    3   33.333333    4.0   0.0     1
2     0    4   26.666667    5.0   0.0     0
3     0    6   26.666667    7.0   0.0     1
4     1    2   14.285714    7.0   0.0     0
5     1    3   14.285714    6.0   0.0     1
6     1    4   17.142857    5.0   0.0     0
7     1    5   17.142857    4.0   0.0     1
8     2    0  100.000000    1.0   0.0     0
9     2    1  100.000000    2.0   0.0     1
10    2    6  200.000000    1.0   0.0     0
11    3    2   50.000000    2.0   0.0     0
12    3    4   50.000000    2.0   0.0     1
13    3    6   25.000000    4.0   0.0     0
                    a          b      c     d
2020-01-01   0.000000   0.000000  100.0   0.0
2020-01-02   0.000000   0.000000    0.0   0.0
2020-01-03  33.333333  14.285714    0.0  50.0
2020-01-04   0.000000   0.000000    0.0  50.0
2020-01-05  26.666667  17.142857    0.0   0.0
2020-01-06  26.666667   0.000000    0.0   0.0
2020-01-07   0.000

In [12]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, fees=0.01) # w/ fees
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, fees=0.01)

    col  idx  size  price  fees  side
0     0    2   1.0    3.0  0.03     0
1     0    3   1.0    4.0  0.04     1
2     0    4   1.0    5.0  0.05     0
3     0    6   1.0    7.0  0.07     1
4     1    2   1.0    7.0  0.07     0
5     1    3   1.0    6.0  0.06     1
6     1    4   1.0    5.0  0.05     0
7     1    5   1.0    4.0  0.04     1
8     2    0   1.0    1.0  0.01     0
9     2    1   1.0    2.0  0.02     1
10    2    6   1.0    1.0  0.01     0
11    3    2   1.0    2.0  0.02     0
12    3    4   1.0    2.0  0.02     1
13    3    6   1.0    4.0  0.04     0
              a    b    c    d
2020-01-01  0.0  0.0  1.0  0.0
2020-01-02  0.0  0.0  0.0  0.0
2020-01-03  1.0  1.0  0.0  1.0
2020-01-04  0.0  0.0  0.0  1.0
2020-01-05  1.0  1.0  0.0  0.0
2020-01-06  1.0  0.0  0.0  0.0
2020-01-07  0.0  0.0  1.0  1.0
                 a       b       c       d
2020-01-01  100.00  100.00   98.99  100.00
2020-01-02  100.00  100.00  100.97  100.00
2020-01-03   96.97   92.93  100.97   97.98
2020-01-04

In [13]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, fixed_fees=1) # w/ fixed fees
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, fixed_fees=1)

    col  idx  size  price  fees  side
0     0    2   1.0    3.0   1.0     0
1     0    3   1.0    4.0   1.0     1
2     0    4   1.0    5.0   1.0     0
3     0    6   1.0    7.0   1.0     1
4     1    2   1.0    7.0   1.0     0
5     1    3   1.0    6.0   1.0     1
6     1    4   1.0    5.0   1.0     0
7     1    5   1.0    4.0   1.0     1
8     2    0   1.0    1.0   1.0     0
9     2    1   1.0    2.0   1.0     1
10    2    6   1.0    1.0   1.0     0
11    3    2   1.0    2.0   1.0     0
12    3    4   1.0    2.0   1.0     1
13    3    6   1.0    4.0   1.0     0
              a    b    c    d
2020-01-01  0.0  0.0  1.0  0.0
2020-01-02  0.0  0.0  0.0  0.0
2020-01-03  1.0  1.0  0.0  1.0
2020-01-04  0.0  0.0  0.0  1.0
2020-01-05  1.0  1.0  0.0  0.0
2020-01-06  1.0  0.0  0.0  0.0
2020-01-07  0.0  0.0  1.0  1.0
                a      b     c      d
2020-01-01  100.0  100.0  98.0  100.0
2020-01-02  100.0  100.0  99.0  100.0
2020-01-03   96.0   92.0  99.0   97.0
2020-01-04   99.0   97.0  99.0

In [14]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, slippage=0.01) # w/ slippage
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, slippage=0.01)

    col  idx  size  price  fees  side
0     0    2   1.0   3.03   0.0     0
1     0    3   1.0   3.96   0.0     1
2     0    4   1.0   5.05   0.0     0
3     0    6   1.0   6.93   0.0     1
4     1    2   1.0   7.07   0.0     0
5     1    3   1.0   5.94   0.0     1
6     1    4   1.0   5.05   0.0     0
7     1    5   1.0   3.96   0.0     1
8     2    0   1.0   1.01   0.0     0
9     2    1   1.0   1.98   0.0     1
10    2    6   1.0   1.01   0.0     0
11    3    2   1.0   2.02   0.0     0
12    3    4   1.0   1.98   0.0     1
13    3    6   1.0   4.04   0.0     0
              a    b    c    d
2020-01-01  0.0  0.0  1.0  0.0
2020-01-02  0.0  0.0  0.0  0.0
2020-01-03  1.0  1.0  0.0  1.0
2020-01-04  0.0  0.0  0.0  1.0
2020-01-05  1.0  1.0  0.0  0.0
2020-01-06  1.0  0.0  0.0  0.0
2020-01-07  0.0  0.0  1.0  1.0
                 a       b       c       d
2020-01-01  100.00  100.00   98.99  100.00
2020-01-02  100.00  100.00  100.97  100.00
2020-01-03   96.97   92.93  100.97   97.98
2020-01-04

## from_orders

In [15]:
order_size = pd.DataFrame.vbt.tseries.empty_like(price, 1) # buy 1 share every day
big_order_size = pd.DataFrame.vbt.tseries.empty_like(big_price, 1)

In [16]:
portfolio = vbt.Portfolio.from_orders(price['a'], order_size['a'])
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_orders(big_price.iloc[:, 0], big_order_size.iloc[:, 0])

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    2   1.0    3.0   0.0     0
3    0    3   1.0    4.0   0.0     0
4    0    4   1.0    5.0   0.0     0
5    0    5   1.0    6.0   0.0     0
6    0    6   1.0    7.0   0.0     0
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64
2020-01-01    99.0
2020-01-02    97.0
2020-01-03    94.0
2020-01-04    90.0
2020-01-05    85.0
2020-01-06    79.0
2020-01-07    72.0
Name: a, dtype: float64
9.61 ms ± 711 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
portfolio = vbt.Portfolio.from_orders(price, order_size)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_orders(big_price, big_order_size)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0   0.0     0
1     0    1   1.0    2.0   0.0     0
2     0    2   1.0    3.0   0.0     0
3     0    3   1.0    4.0   0.0     0
4     0    4   1.0    5.0   0.0     0
5     0    5   1.0    6.0   0.0     0
6     0    6   1.0    7.0   0.0     0
7     1    0   1.0    9.0   0.0     0
8     1    1   1.0    8.0   0.0     0
9     1    2   1.0    7.0   0.0     0
10    1    3   1.0    6.0   0.0     0
11    1    4   1.0    5.0   0.0     0
12    1    5   1.0    4.0   0.0     0
13    1    6   1.0    3.0   0.0     0
14    2    0   1.0    1.0   0.0     0
15    2    1   1.0    2.0   0.0     0
16    2    2   1.0    3.0   0.0     0
17    2    3   1.0    4.0   0.0     0
18    2    4   1.0    3.0   0.0     0
19    2    5   1.0    2.0   0.0     0
20    2    6   1.0    1.0   0.0     0
21    3    0   1.0    4.0   0.0     0
22    3    1   1.0    3.0   0.0     0
23    3    2   1.0    2.0   0.0     0
24    3    3   1.0    1.0   0.0     0
25    3    4

In [18]:
portfolio = vbt.Portfolio.from_orders(price, order_size, is_target=True) # target size
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_orders(big_price, big_order_size, is_target=True)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    1    0   1.0    9.0   0.0     0
2    2    0   1.0    1.0   0.0     0
3    3    0   1.0    4.0   0.0     0
              a    b    c    d
2020-01-01  1.0  1.0  1.0  1.0
2020-01-02  1.0  1.0  1.0  1.0
2020-01-03  1.0  1.0  1.0  1.0
2020-01-04  1.0  1.0  1.0  1.0
2020-01-05  1.0  1.0  1.0  1.0
2020-01-06  1.0  1.0  1.0  1.0
2020-01-07  1.0  1.0  1.0  1.0
               a     b     c     d
2020-01-01  99.0  91.0  99.0  96.0
2020-01-02  99.0  91.0  99.0  96.0
2020-01-03  99.0  91.0  99.0  96.0
2020-01-04  99.0  91.0  99.0  96.0
2020-01-05  99.0  91.0  99.0  96.0
2020-01-06  99.0  91.0  99.0  96.0
2020-01-07  99.0  91.0  99.0  96.0
56.6 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## from_order_func

In [19]:
fees = 0.01
fixed_fees = 1
slippage = 0.01

@njit
def order_func_nb(col, i, run_cash, run_shares, price):
    return vbt.portfolio.nb.Order(-i if i % 2 == 0 else i, price[i, col], fees, fixed_fees, slippage)

portfolio = vbt.Portfolio.from_order_func(price['a'], order_func_nb, price.values)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_order_func(big_price.iloc[:, 0], order_func_nb, big_price.values)

   col  idx  size  price    fees  side
0    0    1   1.0   2.02  1.0202     0
1    0    2   1.0   2.97  1.0297     1
2    0    3   3.0   4.04  1.1212     0
3    0    4   3.0   4.95  1.1485     1
4    0    5   5.0   6.06  1.3030     0
5    0    6   5.0   6.93  1.3465     1
2020-01-01    0.0
2020-01-02    1.0
2020-01-03    0.0
2020-01-04    3.0
2020-01-05    0.0
2020-01-06    5.0
2020-01-07    0.0
Name: a, dtype: float64
2020-01-01    100.0000
2020-01-02     96.9598
2020-01-03     98.9001
2020-01-04     85.6589
2020-01-05     99.3604
2020-01-06     67.7574
2020-01-07    101.0609
Name: a, dtype: float64
6 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
portfolio = vbt.Portfolio.from_order_func(price, order_func_nb, price.values)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_order_func(big_price, order_func_nb, big_price.values)

    col  idx  size  price    fees  side
0     0    1   1.0   2.02  1.0202     0
1     0    2   1.0   2.97  1.0297     1
2     0    3   3.0   4.04  1.1212     0
3     0    4   3.0   4.95  1.1485     1
4     0    5   5.0   6.06  1.3030     0
5     0    6   5.0   6.93  1.3465     1
6     1    1   1.0   8.08  1.0808     0
7     1    2   1.0   6.93  1.0693     1
8     1    3   3.0   6.06  1.1818     0
9     1    4   3.0   4.95  1.1485     1
10    1    5   5.0   4.04  1.2020     0
11    1    6   5.0   2.97  1.1485     1
12    2    1   1.0   2.02  1.0202     0
13    2    2   1.0   2.97  1.0297     1
14    2    3   3.0   4.04  1.1212     0
15    2    4   3.0   2.97  1.0891     1
16    2    5   5.0   2.02  1.1010     0
17    2    6   5.0   0.99  1.0495     1
18    3    1   1.0   3.03  1.0303     0
19    3    2   1.0   1.98  1.0198     1
20    3    3   3.0   1.01  1.0303     0
21    3    4   3.0   1.98  1.0594     1
22    3    5   5.0   3.03  1.1515     0
23    3    6   5.0   3.96  1.1980     1


## Indexing

In [21]:
levy_alpha = 3.
risk_free = 0.01
required_return = 0.1
cutoff = 0.05
factor_returns = price['a'] * 0.9
big_factor_returns = big_price[0] * 0.9

portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, fees=0.01, freq='1 day', year_freq='252 days',
    levy_alpha=levy_alpha, risk_free=risk_free, required_return=required_return, 
    cutoff=cutoff, factor_returns=factor_returns)
print(portfolio.main_price.shape)

big_portfolio = vbt.Portfolio.from_signals(
    big_price, big_entries, big_exits, fees=0.01, freq='1 day', year_freq='252 days',
    levy_alpha=levy_alpha, risk_free=risk_free, required_return=required_return, 
    cutoff=cutoff, factor_returns=big_factor_returns)
print(big_portfolio.main_price.shape)

(7, 4)
(1000, 1000)


In [22]:
print(portfolio.main_price)
print(portfolio.init_capital)
print(portfolio.orders.records)
print(portfolio.cash)
print(portfolio.shares)
print(portfolio.freq)
print(portfolio.year_freq)
print(portfolio.levy_alpha)
print(portfolio.risk_free)
print(portfolio.required_return)
print(portfolio.cutoff)
print(portfolio.factor_returns)

              a    b    c    d
2020-01-01  1.0  9.0  1.0  4.0
2020-01-02  2.0  8.0  2.0  3.0
2020-01-03  3.0  7.0  3.0  2.0
2020-01-04  4.0  6.0  4.0  1.0
2020-01-05  5.0  5.0  3.0  2.0
2020-01-06  6.0  4.0  2.0  3.0
2020-01-07  7.0  3.0  1.0  4.0
a    100
b    100
c    100
d    100
dtype: int64
    col  idx        size  price      fees  side
0     0    2   33.003300    3.0  0.990099     0
1     0    3   33.003300    4.0  1.320132     1
2     0    4   25.879816    5.0  1.293991     0
3     0    6   25.879816    7.0  1.811587     1
4     1    2   14.144272    7.0  0.990099     0
5     1    3   14.144272    6.0  0.848656     1
6     1    4   16.637024    5.0  0.831851     0
7     1    5   16.637024    4.0  0.665481     1
8     2    0   99.009901    1.0  0.990099     0
9     2    1   99.009901    2.0  1.980198     1
10    2    6  194.098618    1.0  1.940986     0
11    3    2   49.504950    2.0  0.990099     0
12    3    4   49.504950    2.0  0.990099     1
13    3    6   24.262327    4.0

In [23]:
print(portfolio['a'].main_price)
print(portfolio['a'].init_capital)
print(portfolio['a'].orders.records)
print(portfolio['a'].cash)
print(portfolio['a'].shares)
print(portfolio['a'].freq)
print(portfolio['a'].year_freq)
print(portfolio['a'].levy_alpha)
print(portfolio['a'].risk_free)
print(portfolio['a'].required_return)
print(portfolio['a'].cutoff)
print(portfolio['a'].factor_returns)

2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64
100
   col  idx       size  price      fees  side
0    0    2  33.003300    3.0  0.990099     0
1    0    3  33.003300    4.0  1.320132     1
2    0    4  25.879816    5.0  1.293991     0
3    0    6  25.879816    7.0  1.811587     1
2020-01-01    100.000000
2020-01-02    100.000000
2020-01-03      0.000000
2020-01-04    130.693069
2020-01-05      0.000000
2020-01-06      0.000000
2020-01-07    179.347123
Name: a, dtype: float64
2020-01-01     0.000000
2020-01-02     0.000000
2020-01-03    33.003300
2020-01-04     0.000000
2020-01-05    25.879816
2020-01-06    25.879816
2020-01-07     0.000000
Name: a, dtype: float64
1 days 00:00:00
252 days 00:00:00
3.0
0.01
0.1
0.05
2020-01-01    0.9
2020-01-02    1.8
2020-01-03    2.7
2020-01-04    3.6
2020-01-05    4.5
2020-01-06    5.4
2020-01-07    6.3
Name: a, dtype: float64


In [24]:
print(portfolio.main_price.iloc[:, 0])
%timeit big_portfolio.main_price.iloc[:, 0]

print(portfolio.iloc[:, 0].main_price)
%timeit big_portfolio.iloc[:, 0].main_price # slower since requires does much more processing

2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64
61.8 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64
6.86 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
print(portfolio.main_price.iloc[:, np.arange(portfolio.main_price.shape[1])])
%timeit big_portfolio.main_price.iloc[:, np.arange(big_portfolio.main_price.shape[1])]

print(portfolio.iloc[:, np.arange(portfolio.main_price.shape[1])].main_price)
%timeit big_portfolio.iloc[:, np.arange(big_portfolio.main_price.shape[1])].main_price

              a    b    c    d
2020-01-01  1.0  9.0  1.0  4.0
2020-01-02  2.0  8.0  2.0  3.0
2020-01-03  3.0  7.0  3.0  2.0
2020-01-04  4.0  6.0  4.0  1.0
2020-01-05  5.0  5.0  3.0  2.0
2020-01-06  6.0  4.0  2.0  3.0
2020-01-07  7.0  3.0  1.0  4.0
3.36 ms ± 64.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
              a    b    c    d
2020-01-01  1.0  9.0  1.0  4.0
2020-01-02  2.0  8.0  2.0  3.0
2020-01-03  3.0  7.0  3.0  2.0
2020-01-04  4.0  6.0  4.0  1.0
2020-01-05  5.0  5.0  3.0  2.0
2020-01-06  6.0  4.0  2.0  3.0
2020-01-07  7.0  3.0  1.0  4.0
31.7 ms ± 967 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
orders = portfolio.orders
big_orders = big_portfolio.orders

print(orders.iloc[:, 0].records)
%timeit big_orders.iloc[:, 0].records

   col  idx       size  price      fees  side
0    0    2  33.003300    3.0  0.990099     0
1    0    3  33.003300    4.0  1.320132     1
2    0    4  25.879816    5.0  1.293991     0
3    0    6  25.879816    7.0  1.811587     1
2.85 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
print(orders.iloc[:, np.arange(portfolio.main_price.shape[1])].records)
%timeit big_orders.iloc[:, np.arange(portfolio.main_price.shape[1])].records

    col  idx        size  price      fees  side
0     0    2   33.003300    3.0  0.990099     0
1     0    3   33.003300    4.0  1.320132     1
2     0    4   25.879816    5.0  1.293991     0
3     0    6   25.879816    7.0  1.811587     1
4     1    2   14.144272    7.0  0.990099     0
5     1    3   14.144272    6.0  0.848656     1
6     1    4   16.637024    5.0  0.831851     0
7     1    5   16.637024    4.0  0.665481     1
8     2    0   99.009901    1.0  0.990099     0
9     2    1   99.009901    2.0  1.980198     1
10    2    6  194.098618    1.0  1.940986     0
11    3    2   49.504950    2.0  0.990099     0
12    3    4   49.504950    2.0  0.990099     1
13    3    6   24.262327    4.0  0.970493     0
3.61 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
print(vbt.Portfolio.from_signals(price, entries, exits, levy_alpha=1)['a'].levy_alpha)
print(vbt.Portfolio.from_signals(price, entries, exits, levy_alpha=np.arange(price.shape[1]))['b'].levy_alpha)

1
1


In [29]:
print(vbt.Portfolio.from_signals(price, entries, exits, factor_returns=1)['a'].factor_returns)
print(vbt.Portfolio.from_signals(price, entries, exits, factor_returns=price['a'])['b'].factor_returns)
print(vbt.Portfolio.from_signals(price, entries, exits, factor_returns=price)['a'].factor_returns)

1
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
2020-01-06    6.0
2020-01-07    7.0
Name: a, dtype: float64


## Records

In [30]:
print(portfolio['a'].orders.records)
print(portfolio.orders['a'].records)
print(portfolio.orders.records)

%timeit big_portfolio.orders.records

   col  idx       size  price      fees  side
0    0    2  33.003300    3.0  0.990099     0
1    0    3  33.003300    4.0  1.320132     1
2    0    4  25.879816    5.0  1.293991     0
3    0    6  25.879816    7.0  1.811587     1
   col  idx       size  price      fees  side
0    0    2  33.003300    3.0  0.990099     0
1    0    3  33.003300    4.0  1.320132     1
2    0    4  25.879816    5.0  1.293991     0
3    0    6  25.879816    7.0  1.811587     1
    col  idx        size  price      fees  side
0     0    2   33.003300    3.0  0.990099     0
1     0    3   33.003300    4.0  1.320132     1
2     0    4   25.879816    5.0  1.293991     0
3     0    6   25.879816    7.0  1.811587     1
4     1    2   14.144272    7.0  0.990099     0
5     1    3   14.144272    6.0  0.848656     1
6     1    4   16.637024    5.0  0.831851     0
7     1    5   16.637024    4.0  0.665481     1
8     2    0   99.009901    1.0  0.990099     0
9     2    1   99.009901    2.0  1.980198     1
10    2    6

In [31]:
print(portfolio['a'].trades.records)
print(portfolio.trades['a'].records)
print(portfolio.trades.records)

%timeit big_portfolio.trades.records

   col  idx       size  open_idx  open_price  open_fees  close_idx  \
0    0    3  33.003300         2         3.0   0.990099          3   
1    0    6  25.879816         4         5.0   1.293991          6   

   close_price  close_fees        pnl    return  status  position_idx  
0          4.0    1.320132  30.693069  0.306931       1             0  
1          7.0    1.811587  48.654054  0.372277       1             1  
   col  idx       size  open_idx  open_price  open_fees  close_idx  \
0    0    3  33.003300         2         3.0   0.990099          3   
1    0    6  25.879816         4         5.0   1.293991          6   

   close_price  close_fees        pnl    return  status  position_idx  
0          4.0    1.320132  30.693069  0.306931       1             0  
1          7.0    1.811587  48.654054  0.372277       1             1  
   col  idx        size  open_idx  open_price  open_fees  close_idx  \
0    0    3   33.003300         2         3.0   0.990099          3   
1   

In [32]:
print(portfolio['a'].positions.records)
print(portfolio.positions['a'].records)
print(portfolio.positions.records)

%timeit big_portfolio.positions.records

   col  idx       size  open_idx  open_price  open_fees  close_idx  \
0    0    3  33.003300         2         3.0   0.990099          3   
1    0    6  25.879816         4         5.0   1.293991          6   

   close_price  close_fees        pnl    return  status  
0          4.0    1.320132  30.693069  0.306931       1  
1          7.0    1.811587  48.654054  0.372277       1  
   col  idx       size  open_idx  open_price  open_fees  close_idx  \
0    0    3  33.003300         2         3.0   0.990099          3   
1    0    6  25.879816         4         5.0   1.293991          6   

   close_price  close_fees        pnl    return  status  
0          4.0    1.320132  30.693069  0.306931       1  
1          7.0    1.811587  48.654054  0.372277       1  
   col  idx        size  open_idx  open_price  open_fees  close_idx  \
0    0    3   33.003300         2         3.0   0.990099          3   
1    0    6   25.879816         4         5.0   1.293991          6   
2    1    3   14.

## Equity

In [33]:
print(portfolio['a'].equity)
print(portfolio.equity)

%timeit big_portfolio.equity

2020-01-01    100.000000
2020-01-02    100.000000
2020-01-03     99.009901
2020-01-04    130.693069
2020-01-05    129.399079
2020-01-06    155.278894
2020-01-07    179.347123
Name: a, dtype: float64
                     a           b           c           d
2020-01-01  100.000000  100.000000   99.009901  100.000000
2020-01-02  100.000000  100.000000  196.039604  100.000000
2020-01-03   99.009901   99.009901  196.039604   99.009901
2020-01-04  130.693069   84.016973  196.039604   49.504950
2020-01-05  129.399079   83.185122  196.039604   98.019802
2020-01-06  155.278894   65.882617  196.039604   98.019802
2020-01-07  179.347123   65.882617  194.098618   97.049309
7.97 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
print(portfolio['a'].final_equity)
print(portfolio.final_equity)

%timeit big_portfolio.final_equity

179.347122831095
a    179.347123
b     65.882617
c    194.098618
d     97.049309
dtype: float64
12.7 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
print(portfolio['a'].peak_equity)
print(portfolio.peak_equity)

%timeit big_portfolio.peak_equity

179.347122831095
a    179.347123
b    100.000000
c    196.039604
d    100.000000
dtype: float64
29.2 ms ± 7.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
print(portfolio['a'].dip_equity)
print(portfolio.dip_equity)

%timeit big_portfolio.dip_equity

99.00990099009903
a    99.009901
b    65.882617
c    99.009901
d    49.504950
dtype: float64
23.6 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
print(portfolio['a'].total_profit)
print(portfolio.total_profit)

%timeit big_portfolio.total_profit

79.34712283109499
a    79.347123
b   -34.117383
c    94.098618
d    -2.950691
dtype: float64
10.6 ms ± 1e+03 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Drawdown

In [38]:
print(portfolio['a'].drawdown)
print(portfolio.drawdown)

%timeit big_portfolio.drawdown

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03   -0.009901
2020-01-04    0.000000
2020-01-05   -0.009901
2020-01-06    0.000000
2020-01-07    0.000000
Name: a, dtype: float64
                   a         b         c         d
2020-01-01  0.000000  0.000000  0.000000  0.000000
2020-01-02  0.000000  0.000000  0.000000  0.000000
2020-01-03 -0.009901 -0.009901  0.000000 -0.009901
2020-01-04  0.000000 -0.159830  0.000000 -0.504950
2020-01-05 -0.009901 -0.168149  0.000000 -0.019802
2020-01-06  0.000000 -0.341174  0.000000 -0.019802
2020-01-07  0.000000 -0.341174 -0.009901 -0.029507
24.5 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
print(portfolio['a'].max_drawdown)
print(portfolio.max_drawdown)

%timeit big_portfolio.max_drawdown

-0.00990099009901002
a   -0.009901
b   -0.341174
c   -0.009901
d   -0.504950
dtype: float64
28.3 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
print(portfolio['a'].drawdowns.records)
print(portfolio.drawdowns['a'].records)
print(portfolio.drawdowns.records)

%timeit big_portfolio.drawdowns.records

   col  idx  start_idx  valley_idx  end_idx  status
0    0    3          1           2        3       1
1    0    5          3           4        5       1
   col  idx  start_idx  valley_idx  end_idx  status
0    0    3          1           2        3       1
1    0    5          3           4        5       1
   col  idx  start_idx  valley_idx  end_idx  status
0    0    3          1           2        3       1
1    0    5          3           4        5       1
2    1    6          1           5        6       0
3    2    6          5           6        6       0
4    3    6          1           3        6       0
17.8 ms ± 874 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Returns

In [41]:
print(portfolio['a'].buy_and_hold_return)
print(portfolio.buy_and_hold_return)

%timeit big_portfolio.buy_and_hold_return

6.0
a    6.000000
b   -0.666667
c    0.000000
d    0.000000
dtype: float64
13.3 ms ± 279 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
print(portfolio['a'].returns)
print(portfolio.returns)

%timeit big_portfolio.equity.pct_change()
%timeit big_portfolio.returns # numba helps a lot

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03   -0.009901
2020-01-04    0.320000
2020-01-05   -0.009901
2020-01-06    0.200000
2020-01-07    0.155000
Name: a, dtype: float64
                   a         b         c         d
2020-01-01  0.000000  0.000000 -0.009901  0.000000
2020-01-02  0.000000  0.000000  0.980000  0.000000
2020-01-03 -0.009901 -0.009901  0.000000 -0.009901
2020-01-04  0.320000 -0.151429  0.000000 -0.500000
2020-01-05 -0.009901 -0.009901  0.000000  0.980000
2020-01-06  0.200000 -0.208000  0.000000  0.000000
2020-01-07  0.155000  0.000000 -0.009901 -0.009901
356 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
18 ms ± 932 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
print(portfolio['a'].daily_returns)
print(portfolio.daily_returns)

%timeit big_portfolio.daily_returns

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03   -0.009901
2020-01-04    0.320000
2020-01-05   -0.009901
2020-01-06    0.200000
2020-01-07    0.155000
Name: a, dtype: float64
                   a         b         c         d
2020-01-01  0.000000  0.000000 -0.009901  0.000000
2020-01-02  0.000000  0.000000  0.980000  0.000000
2020-01-03 -0.009901 -0.009901  0.000000 -0.009901
2020-01-04  0.320000 -0.151429  0.000000 -0.500000
2020-01-05 -0.009901 -0.009901  0.000000  0.980000
2020-01-06  0.200000 -0.208000  0.000000  0.000000
2020-01-07  0.155000  0.000000 -0.009901 -0.009901
20.7 ms ± 4.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
print(portfolio['a'].annual_returns)
print(portfolio.annual_returns)

%timeit big_portfolio.annual_returns

2020-01-01    0.793471
Freq: 252D, Name: a, dtype: float64
                   a         b         c         d
2020-01-01  0.793471 -0.341174  0.940986 -0.029507
31.5 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
print(portfolio['a'].cumulative_returns)
print(portfolio.cumulative_returns)

%timeit big_portfolio.cumulative_returns

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03   -0.009901
2020-01-04    0.306931
2020-01-05    0.293991
2020-01-06    0.552789
2020-01-07    0.793471
Name: a, dtype: float64
                   a         b         c         d
2020-01-01  0.000000  0.000000 -0.009901  0.000000
2020-01-02  0.000000  0.000000  0.960396  0.000000
2020-01-03 -0.009901 -0.009901  0.960396 -0.009901
2020-01-04  0.306931 -0.159830  0.960396 -0.504950
2020-01-05  0.293991 -0.168149  0.960396 -0.019802
2020-01-06  0.552789 -0.341174  0.960396 -0.019802
2020-01-07  0.793471 -0.341174  0.940986 -0.029507
35.5 ms ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
print(portfolio['a'].total_return)
print(portfolio.total_return)

%timeit big_portfolio.total_return

0.7934712283109495
a    0.793471
b   -0.341174
c    0.940986
d   -0.029507
dtype: float64
23.7 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
print(empyrical.annual_return(portfolio['a'].returns.values))

print(portfolio['a'].annualized_return)
print(portfolio.annualized_return)

%timeit big_portfolio.annualized_return

1358310008.2882464
1358310008.2882464
a    1.358310e+09
b   -9.999997e-01
c    2.337803e+10
d   -6.598049e-01
dtype: float64
23.4 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
print(empyrical.annual_volatility(portfolio['a'].returns.values, alpha=levy_alpha))

print(portfolio['a'].annualized_volatility)
print(portfolio.annualized_volatility)

%timeit big_portfolio.annualized_volatility

0.8367476434836345
0.8367476434836347
a    0.836748
b    0.552128
c    2.347677
d    2.801529
dtype: float64
26.6 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
print(empyrical.calmar_ratio(portfolio['a'].returns.values))

print(portfolio['a'].calmar_ratio)
print(portfolio.calmar_ratio)

%timeit big_portfolio.calmar_ratio

137189310837.11166
137189310837.11122
a    1.371893e+11
b   -2.931056e+00
c    2.361181e+12
d   -1.306672e+00
dtype: float64
35.1 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
print(empyrical.omega_ratio(portfolio['a'].returns.values, risk_free=risk_free, required_return=required_return))

print(portfolio['a'].omega_ratio)
print(portfolio.omega_ratio)

%timeit big_portfolio.omega_ratio

10.500918435712794
10.500918435712794
a    10.500918
b     0.000000
c    11.814325
d     1.665811
dtype: float64
30.6 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
print(empyrical.sharpe_ratio(portfolio['a'].returns.values, risk_free=risk_free))

print(portfolio['a'].sharpe_ratio)
print(portfolio.sharpe_ratio)

%timeit big_portfolio.sharpe_ratio

10.017916955773694
10.017916955773693
a    10.017917
b   -11.654631
c     5.431471
d     1.995074
dtype: float64
28.1 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
print(empyrical.downside_risk(portfolio['a'].returns.values, required_return=required_return))

print(portfolio['a'].downside_risk)
print(portfolio.downside_risk)

%timeit big_portfolio.downside_risk

1.2608062456148732
1.2608062456148732
a    1.260806
b    2.764150
c    1.519747
d    3.861299
dtype: float64
28.4 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
print(empyrical.sortino_ratio(portfolio['a'].returns.values, required_return=required_return))

print(portfolio['a'].sortino_ratio)
print(portfolio.sortino_ratio)

%timeit big_portfolio.sortino_ratio

-1.2792380214948547
-1.2792380214948547
a    -1.279238
b   -14.055787
c     6.163609
d    -2.235743
dtype: float64
29.8 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
print(empyrical.excess_sharpe(portfolio['a'].returns.values, factor_returns))

print(portfolio['a'].information_ratio)
print(portfolio.information_ratio)

%timeit big_portfolio.information_ratio

-1.8641444566093555
-1.8641444566093555
a   -1.864144
b   -1.847420
c   -1.631279
d   -1.840194
dtype: float64
28 ms ± 2.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
print(empyrical.beta(portfolio['a'].returns.values, factor_returns))

print(portfolio['a'].beta)
print(portfolio.beta)

%timeit big_portfolio.beta

nan
0.03432539682539679
a    0.034325
b   -0.016508
c   -0.077778
d    0.038103
dtype: float64
32.7 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
print(empyrical.alpha(portfolio['a'].returns.values, factor_returns, risk_free=risk_free))

print(portfolio['a'].alpha)
print(portfolio.alpha)

%timeit big_portfolio.alpha

nan
-0.999962442278867
a   -9.999624e-01
b   -7.108907e-01
c    2.103372e+37
d   -1.000000e+00
dtype: float64
38.7 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
print(empyrical.tail_ratio(portfolio['a'].returns.values))

print(portfolio['a'].tail_ratio)
print(portfolio.tail_ratio)

%timeit big_portfolio.tail_ratio

28.68399999999992
28.68399999999992
a    28.684000
b     0.000000
c    69.286000
d     1.943506
dtype: float64
66.2 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
print(empyrical.value_at_risk(portfolio['a'].returns.values[1:], cutoff=cutoff))

print(portfolio['a'].value_at_risk)
print(portfolio.value_at_risk)

%timeit big_portfolio.value_at_risk

-0.009900990099009938
-0.00990099009900992
a   -0.009901
b   -0.191029
c   -0.009901
d   -0.352970
dtype: float64
45.9 ms ± 6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
print(empyrical.conditional_value_at_risk(portfolio['a'].returns.values, cutoff=cutoff))

print(portfolio['a'].conditional_value_at_risk)
print(portfolio.conditional_value_at_risk)

%timeit big_portfolio.conditional_value_at_risk

-0.00990099009901002
-0.00990099009901002
a   -0.009901
b   -0.208000
c   -0.009901
d   -0.500000
dtype: float64
58.4 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
print(empyrical.capture(portfolio['a'].returns.values, factor_returns))

print(portfolio['a'].capture)
print(portfolio.capture)

%timeit big_portfolio.capture

8.52073937232888e-149
8.52073937232888e-149
a    8.520739e-149
b   -6.273043e-158
c    1.466514e-147
d   -4.138985e-158
dtype: float64
33.2 ms ± 16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
print(empyrical.up_capture(portfolio['a'].returns.values, factor_returns))

print(portfolio['a'].up_capture)
print(portfolio.up_capture)

%timeit big_portfolio.up_capture

8.52073937232888e-149
8.52073937232888e-149
a    8.520739e-149
b   -6.273043e-158
c    1.466514e-147
d   -4.138985e-158
dtype: float64
31.3 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
print(empyrical.down_capture(portfolio['a'].returns.values, factor_returns))

print(portfolio['a'].down_capture)
print(portfolio.down_capture)

%timeit big_portfolio.down_capture

nan
nan
a   NaN
b   NaN
c   NaN
d   NaN
dtype: float64
24.7 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Stats

In [63]:
print(portfolio['a'].stats)

%timeit big_portfolio[0].stats

Start                     2020-01-01 00:00:00
End                       2020-01-07 00:00:00
Duration                      7 days 00:00:00
Time in Position [%]                  42.8571
Total Profit                          79.3471
Total Return [%]                      79.3471
Buy & Hold Return [%]                     600
Max. Drawdown [%]                    0.990099
Avg. Drawdown [%]                    0.990099
Max. Drawdown Duration        2 days 00:00:00
Avg. Drawdown Duration        2 days 00:00:00
Num. Trades                                 2
Win Rate [%]                              100
Best Trade [%]                        37.2277
Worst Trade [%]                       30.6931
Avg. Trade [%]                        33.9604
Max. Trade Duration           2 days 00:00:00
Avg. Trade Duration           1 days 12:00:00
Expectancy                            39.6736
SQN                                   4.41775
Sharpe Ratio                          10.0179
Sortino Ratio                     